In [40]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
#from fake_useragent import UserAgent
from selenium.webdriver.common.by import By
from selenium import webdriver

driver = webdriver.Chrome('../chromedriver.exe')


/var/folders/7p/4qg7xngs2p9cwv9rq97048fc0000gn/T/ipykernel_60875/1547704011.py:8: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('../chromedriver.exe')


In [41]:
#days = ['mon', 'tue', 'wed', 'thu','fri','sat','sun','dailyPlus', 'new', 'finish']
day = 'tue'
URL = f"https://comic.naver.com/webtoon?tab={day}"
df = pd.DataFrame(columns=['day','titleName', 'titleID', 'authorName', 'authorID','rating', 'url'])

driver.get(URL)

In [42]:
soup = BeautifulSoup(driver.page_source, 'html.parser')
data = soup.find_all('div', attrs={"class" : "ContentList__info_area--bXx7h"})

In [25]:
rating = data[31].findAll('span')[0].text
print(rating)

휴재


In [43]:
soup = BeautifulSoup(driver.page_source, 'html.parser')
data = soup.find_all('div', attrs={"class" : "ContentList__info_area--bXx7h"})

for i in range(len(data)):
    titleName = data[i].findAll('a')[0].findAll('span')[-1].text
            
    titleID = data[i].find('a')['href'].split('=')[-1]
    try:
        authorID = data[i].findAll('a')[1]['href'].split('=')[1]
    except IndexError:
        authorID = data[i].findAll('a')[0]['href'].split('=')[1]
            
    try:
        authorName = data[i].findAll('a')[1].text
    except IndexError:
        authorName = data[i].findAll('a')[0].text
            
    if data[i].findAll('span')[0].text == 'UP' or data[i].findAll('span')[0].text == '휴재':
        rating = data[i].findAll('span')[5].text
    else:
        rating = data[i].findAll('span')[4].text
    
    url = f"https://comic.naver.com{data[i].findAll('a')[0]['href']}"
            
            # df = pd.DataFrame(columns=['day','title', 'authorID', 'authorName','rating', 'url'])
    df = pd.concat([df, pd.DataFrame([[day, titleName, titleID, authorName, authorID, rating, url]],
                                    columns = ['day','titleName', 'titleID', 'authorName', 'authorID','rating', 'url'])], ignore_index=True)

In [44]:
print(df)

    day     titleName titleID      authorName authorID rating   
0   tue           김부장  783053  박태준 만화회사 / 정종택   783053   9.42  \
1   tue     내가 키운 S급들  784248  seri / 비완 / 근서   784248   9.95   
2   tue        마루는 강쥐  796152              모죠   325321   9.98   
3   tue      대학원 탈출일지  790713              요다   353932   9.98   
4   tue          여신강림  703846             야옹이   316796   9.34   
..  ...           ...     ...             ...      ...    ...   
73  tue      달이 사라진 밤  795649             황지음   356004   9.85   
74  tue     찐:종합게임동아리  771018              소장   345445   9.82   
75  tue         AI 유하  800400        박현 / 지홍주   800400   9.84   
76  tue  너의 순정, 나의 순정  797258              정살   356862   9.85   
77  tue         마녀이야기  794671              원재   355601   9.39   

                                                  url  
0   https://comic.naver.com/webtoon/list?titleId=7...  
1   https://comic.naver.com/webtoon/list?titleId=7...  
2   https://comic.naver.com/webtoon/list?titleId=7.

In [38]:
df.index += 1
df.to_csv('webtoon_wed_data.csv', index_label='index')

In [39]:
driver.close()